# GitHub Repository Data from WebScrapping

Github link>>
https://github.com/topics

This project is related with projects posted on different different topics on GITHUB.
We will take each topic from github.
From each topic we find USERNAME,PROJECTNAME,# of STARS,# of Watches Project links.

# using REQUEST lib of python to download data from page

In [1]:
# installing request library
#!pip install requests

In [2]:
import requests

In [4]:
response_obj=requests.get('https://github.com/topics')

In [ ]:
response_obj.status_code

200 means ....request is succesfull

TYPES OF STATUS_CODE.....................
Informational responses (100–199)............
Successful responses (200–299).............
Redirection messages (300–399)...........
Client error responses (400–499)............
Server error responses (500–599)

response_obj.text    

will contain all data in text format

In [7]:
page_content=response_obj.text

In [ ]:
print(page_content[0:2000])
# it is in html format...so lets save it in html format

# Parsing Pages with BeautifulSoup

In [ ]:
!pip install beautifulsoup4 --upgrade --quiet

In [5]:
from bs4 import BeautifulSoup

In [8]:
doc=BeautifulSoup(page_content,'html.parser')

In [9]:
p_tags=doc.find_all('p')

In [10]:
len(p_tags)

67

In [11]:
p_tags[:5]

[<p class="f4 color-fg-muted col-md-6 mx-auto">Browse popular topics on GitHub.</p>,
 <p class="f3 lh-condensed text-center Link--primary mb-0 mt-1">
         Google
       </p>,
 <p class="f5 color-fg-muted text-center mb-0 mt-1">Google LLC is an American multinational technology company that specializes in Internet-related services and products.</p>,
 <p class="f3 lh-condensed text-center Link--primary mb-0 mt-1">
         Vagrant
       </p>,
 <p class="f5 color-fg-muted text-center mb-0 mt-1">Vagrant is an open-source software product for building and maintaining portable virtual software development environments.</p>]

In [12]:
topic_title_tags=doc.find_all('p',{'class':"f3 lh-condensed mb-0 mt-1 Link--primary"})


In [13]:
topic_discription_tags=doc.find_all('p',{'class':"f5 color-fg-muted mb-0 mt-1"})

In [14]:
topic_url_tags=doc.find_all('a',{'class':"d-flex no-underline"})

In [17]:
topic_url_tags[0]['href']

'/topics/3d'

In [18]:
tpoic_link1="https://github.com/"+topic_link[0]['href']

NameError: name 'topic_link' is not defined

In [ ]:
print(tpoic_link1)

Creating Lists

In [19]:
topic_title=[]
for tag in topic_title_tags:
    topic_title.append(tag.text)

In [20]:
topic_discription=[]
for tag in topic_discription_tags:
    topic_discription.append(tag.text.strip())

In [21]:
topic_url=[]
base_url="https://github.com"
for tag in topic_url_tags:
    topic_url.append(base_url+tag['href'].strip())

In [ ]:
topic_url[:5]

converting Lists into dataframe

### Creating a function who will take URL of github topics and return data frame containg all topics posted (on github) , theirs URLs, & discription. 

In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

base_url="https://github.com"
topic_title=[]
topic_discription=[]
topic_url=[]

def get_github_topics(github_topic_url):
    response_obj=requests.get(github_topic_url)
    page_content=response_obj.text
    doc=BeautifulSoup(page_content,'html.parser')
    
    topic_title_tags=doc.find_all('p',{'class':"f3 lh-condensed mb-0 mt-1 Link--primary"})
    topic_discription_tags=doc.find_all('p',{'class':"f5 color-fg-muted mb-0 mt-1"})
    topic_url_tags=doc.find_all('a',{'class':"d-flex no-underline"})
    
    
    for tag in topic_title_tags:
        topic_title.append(tag.text)
    
    for tag in topic_discription_tags:
        topic_discription.append(tag.text.strip())
    
    for tag in topic_url_tags:
        topic_url.append(base_url+tag['href'].strip()) 
        
    return pd.DataFrame(list(zip(topic_title, topic_discription, topic_url)),
              columns=['Title','Discription', 'URL'])

In [39]:
topics_details_df=get_github_topics('https://github.com/topics')

## Saving DATAs to CSV Files

In [34]:
import os
os.mkdir('./WebData')

In [40]:
topics_details_df.to_csv('./WebData/topicsdata.csv',index=None)

## Scrapping another PAGE form given URLs

In [25]:
response=requests.get(topic_url[0]).text

In [26]:
page_doc=BeautifulSoup(response,'html.parser')

In [27]:
name_tags=page_doc.find_all('h3',{'class':"f3 color-fg-muted text-normal lh-condensed"})
star_tags=page_doc.find_all('div',{'class':"d-flex flex-items-start ml-3"})

In [28]:
user_name=[]
repository_name=[]
repository_url=[]
for tag in name_tags:
    user_name.append(tag.find_all('a')[0].text.strip())
    repository_name.append(tag.find_all('a')[1].text.strip())
    repository_url.append("https://github.com" + tag.find_all('a')[1]['href'])

In [29]:
no_of_stars=[]
for tag in star_tags:
    no_of_stars.append(float(tag.find_all('a')[-1].text.strip()[:-1])*1000)

In [30]:
topic_info_df=pd.DataFrame(list(zip(user_name,repository_name,no_of_stars,repository_url)),
                     columns=['user_name','repository_name','no_of_stars','repository_url'])

### putting above things in one function to get info about a particular topic like....3D,ajax...

In [41]:
def get_topic_info(topic_url):
    response=requests.get(topic_url)
    
    if response.status_code !=200:
        raise Exception('falied to load {}'.format(topic_url))
        
    page_doc=BeautifulSoup(response.text,'html.parser')
    name_tags=page_doc.find_all('h3',{'class':"f3 color-fg-muted text-normal lh-condensed"})
    star_tags=page_doc.find_all('div',{'class':"d-flex flex-items-start ml-3"})
    user_name=[]
    repository_name=[]
    repository_url=[]
    for tag in name_tags:
        user_name.append(tag.find_all('a')[0].text.strip())
        repository_name.append(tag.find_all('a')[1].text.strip())
        repository_url.append("https://github.com" + tag.find_all('a')[1]['href'])
    no_of_stars=[]
    for tag in star_tags:
        no_of_stars.append(float(tag.find_all('a')[-1].text.strip()[:-1])*1000)
    return pd.DataFrame(list(zip(user_name,repository_name,no_of_stars,repository_url)),
                     columns=['user_name','repository_name','no_of_stars','repository_url'])

### Creating CSV file of all topics having infomation of USER_NAME, REPOSITORY_NAME, NO_OF_STARS, REPOSITORY_URL...  in WebData folder of current working directory 

In [50]:
import os
for index,row in topics_df.iterrows():
    path_name='./WebData/'+row['Title']+'.csv'
    if not os.path.exists(fname):
        data_df=get_topic_info(row['URL'])
        data_df.to_csv(path_name,index=None)

Exception: falied to load https://github.com/topics/chrome

## documentation 

In [ ]:
Tools Used: Python,os, requests,pandas,BeautifulSoup